In [42]:
import pyopencl as cl #ONLY RUN THIS IF YOU'RE USING GPU

ModuleNotFoundError: No module named 'pyopencl'

In [43]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to /Users/Rgao/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/Rgao/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/Rgao/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [44]:
import pandas as pd
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string

In [45]:
pre_df = pd.read_csv('./data/pre_soccer_replaced.csv')
post_df = pd.read_csv('./data/post_soccer_replaced.csv')

In [51]:
from datetime import datetime
pre_df['pcreated']=[datetime.fromtimestamp(x) for x in pre_df['pcreated']]
post_df['pcreated']=[datetime.fromtimestamp(x) for x in post_df['pcreated']]
pre_df['ccreated']=[datetime.fromtimestamp(x) for x in pre_df['ccreated']]
post_df['ccreated']=[datetime.fromtimestamp(x) for x in post_df['ccreated']]

# 1. I have fix dates 
# 2. Seperate the teams out from title, pickle the files
# 3. Anu -> sentiment analysis
# 4. Sid -> Report starting
# 5. Bog -> front-end, matching

In [48]:
def TokenizeProcess(df):
    word_lemmatizer = WordNetLemmatizer()

    df_tk = pd.DataFrame(columns=['ptitle', 'pscore', 'pid', 'pbody', 'pcreated', 'comment', 'cauthor', 'ccreated'])

    for (ptitle, pscore, pid, pbody, pcreated, comment, cauthor, ccreated) in df.values.tolist():
        # Tokenize
        tokens = word_tokenize(str(comment))

        # Strip punctuation
        punctuation_list = str.maketrans('', '', string.punctuation)
        tokens_strp = [w.translate(punctuation_list) for w in tokens]

        # Remove other non-alphabetic tokens
        words = [word for word in tokens_strp if word.isalpha()]

        # Stop words
        stop_words = set(stopwords.words('english'))
        words = [w for w in words if not w in stop_words]

        # Lemmatize
        # Note: We might need to do this for the brands we want to detect as well
        words_lm = [word_lemmatizer.lemmatize(w) for w in words]
        df_tk = df_tk.append({"ptitle": ptitle, 'pscore':pscore, 'pid':pid, 'pbody':pbody, 'pcreated': pcreated, 'comment': words_lm, 'cauthor': cauthor, 'ccreated': ccreated} ,ignore_index=True) 
    return df_tk

In [49]:
pre_df_tk = TokenizeProcess(pre_df)


KeyboardInterrupt



In [ ]:
post_df_tk = TokenizeProcess(post_df)

In [10]:
pre_df_tk.head(10)

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[If, stop, Sporting, winning, league, least, m...",idek0k,NaN
1,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[I, nt, follow, Liga, NOS, closely, Portuguese...",kuzjaruge,NaN
2,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Go, Porto, get, win, I, look, match, thread, ...",TweakyWatson,NaN
3,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[It, shame, fan, allowed, stadium, I, think, w...",rabbitvinyl,NaN
4,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Juventus, take, easily, porto, fairly, decent...",NunoxGames,NaN
5,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Oh, Cuadrado, injured, Sucks, I, looking, for...",atropicalpenguin,NaN
6,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[Marega, masterclass]",Homygod319,NaN
7,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[McKennie, Bentancur, Rabiot, starting, might,...",jv17fortheW,NaN
8,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[I, fucking, hope, Bernardeschi, nt, start, du...",Handyman2116,NaN
9,[Pre-Match Thread] Porto vs Juventus (Champion...,60,lls3ua,**Porto vs Juventus**\n\nCompetition: UEFA Cha...,1.613589e+09,"[happy, get, watch, goat, play, champion, leag...",90degreeturnsbtw,NaN


In [22]:
post_df.head(10)

,ptitle,pscore,pid,pbody,pcreated,comment,cauthor,ccreated
0,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"**gifs / stats / quotes** *i am a bot, and th...",AutoModerator,1.613541e+09
1,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,[mbappe v FC Barcelona](https://preview.redd.i...,CreateNewAccountsss,1.613541e+09
2,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,good guy Paris Saint-Germain saved barcelona t...,codespyder,1.613541e+09
3,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"on the bright side for barcelona, this is a 50...",TheChiffre,1.613541e+09
4,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,"koeman set it up perfectly for remontada 2.0, ...",SomeIrishFiend,1.613541e+09
5,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,a complete list of every time a team has come ...,gnorrn,1.613541e+09
6,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,i can't remember a barcelona season like this....,MrSteelCity,1.613541e+09
7,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,it's a bad day to have eyes.,razycal970,1.613541e+09
8,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,Paris Saint-Germain having a comfortable lead ...,Sdub4,1.613541e+09
9,Post Match Thread: FC Barcelona 1-4 Paris Sain...,2435,llen1t,#[](#sprite1-p6) vs [](#sprite1-p35)\n\n**Venu...,1.613541e+09,can’t wait to this to be the night which convi...,LosTerminators,1.613541e+09


In [23]:
pre_df_tk.to_csv("./data/pre_soccer_tokenized.csv",index=False)
post_df_tk.to_csv("./data/post_soccer_tokenized.csv",index=False)